## Import dependent libraries

In [24]:
import math
import random
from collections import deque
from pandas import read_csv
from matplotlib import pyplot
from numpy import concatenate, array, asarray
from pandas import DataFrame
from pandas import concat
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
from keras.models import Model, model_from_json
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical, plot_model
from keras.callbacks import ReduceLROnPlateau
import keras.backend as K

## Convert series to supervised learning

In [7]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Ingest Data

In [8]:
def ingest_csv(fileName):
    dataset = read_csv(fileName, header=0)
    values = dataset.values
    # ensure all data is float
    values = values.astype('float32')
    return values

## Encode and Scale the values

In [9]:
# specify the number of lag hours
n_hours = 10
n_features = 2
n_obs = n_hours * n_features

values = ingest_csv('music_data.csv')

# normalize features
scaler = joblib.load('scaler.pkl')
scaled = scaler.fit_transform(values)

In [10]:
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed[0:1:])

    var1(t-10)  var2(t-10)  var3(t-10)  var1(t-9)  var2(t-9)  var3(t-9)  \
10         0.0         0.0         0.0        0.0        0.0   0.011905   

    var1(t-8)  var2(t-8)  var3(t-8)  var1(t-7)    ...     var3(t-3)  \
10        0.0        0.0    0.02381        0.0    ...      0.083333   

    var1(t-2)  var2(t-2)  var3(t-2)  var1(t-1)  var2(t-1)  var3(t-1)  var1(t)  \
10        0.0        0.0   0.095238        0.0        0.0   0.107143      0.0   

     var2(t)   var3(t)  
10  0.032258  0.119048  

[1 rows x 33 columns]


In [11]:
values = reframed.values
n_train_steps = 9000
train = values[:n_train_steps, :]
test = values[n_train_steps:, :]
# split into input and outputs
n_obs = n_hours * n_features

In [22]:
train_X, train_y_notes = train[:, :n_obs], train[:, -n_features]
train_y_length = train[:, -n_features -1]

test_X, test_y_notes = test[:, :n_obs], test[:, -n_features]
test_y_length = test[:, -n_features -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y_notes.shape, test_X.shape, test_y_notes.shape)

(1588, 10, 2) (1588,) (0, 10, 2) (0,)


In [29]:
# load json and create model
params = {'num': 0}
setattr(K, 'params', params)
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

NameError: name 'n_hours' is not defined

In [28]:
loaded_model.summary()

NameError: name 'loaded_model' is not defined

In [15]:
def process_array_for_prediction(testx):
    testx = DataFrame(data = testx)
    testx = testx.values
    testx = testx.astype('float32')
    testx = scaler.transform(testx)
    testx = series_to_supervised(testx, n_hours, 1)
    testx = testx.values
    testx = testx[:, :n_obs]
    testx = testx.reshape((testx.shape[0], n_hours, n_features))
    return testx

In [16]:
# make a prediction

testx = [[60, 25456],
[59, 14880],
[62, 25820],
[60, 21361],
[59, 21153],
[60, 26320],
[58, 20404],
[57, 86203],
[59, 20645],
[57, 19400],
[60, 26310]]

input_values = testx

testx = process_array_for_prediction(testx)

ValueError: operands could not be broadcast together with shapes (11,2) (3,) (11,2) 

In [17]:
prediction = loaded_model.predict(testx)
prediction = asarray(prediction).ravel().reshape(-1,2)
print(prediction)

NameError: name 'loaded_model' is not defined

In [4]:
prediction = scaler.inverse_transform(prediction)
prediction = prediction.astype('int')

NameError: name 'scaler' is not defined

In [5]:
print("INPUT_VALUES:")
print(input_values)
print("PREDICTION:")
print(prediction[0])
print("Note:", prediction[0][0], "Length:", prediction[0][1])

INPUT_VALUES:
[[60, 25456], [59, 14880], [62, 25820], [60, 21361], [59, 21153], [60, 26320], [58, 20404], [57, 86203], [59, 20645], [57, 19400], [60, 26310]]
PREDICTION:


NameError: name 'prediction' is not defined